In [2]:
import mmcv
import numpy as np
import cv2
from tqdm import tqdm_notebook as tqdm
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
ann = mmcv.load("./result/post_nms.pkl")
ret = {}
names = ann.keys()
for name in tqdm(names):
    ret[name] = {}
    result = ann[name]
    segs = result['vis']
    bboxes = result["bbox"]
    objects = []
    for cls in range(18):
        curr_class = CLASSES[cls]
        bbox_cls = np.array(bboxes[cls])
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for box, rle in zip(bbox_cls, seg_cls):
                obj = {}
                rle = np.array(rle)
                _,_,_,_, score = box
                poly = cv2.minAreaRect(rle)
                obj["poly"] = poly
                obj["category"] = curr_class
                obj["score"] = score
                objects.append(obj)
    ret[name]["objects"] = objects
mmcv.dump(ret, "./result/eval_analy.pkl") 
                

In [3]:
# get all images containing given categories, select one at random
import sys
sys.path.append('D:\API\cocoapi\PythonAPI')  # 你下载的 cocoapi 所在路径
import skimage.io as io
from pycocotools.coco import COCO   # 载入 cocoz
%matplotlib inline
coco=COCO("/home/xfr/rssid/data/annotation/annos_rscup_val.json")
imgIds = coco.getImgIds()
# imgIds = coco.getImgIds(imgIds=[335328])
ret = {}
for imgid in tqdm(imgIds):
    info = coco.loadImgs([imgid])[0]
    name = info['file_name'].split(".")[0]
    ret[name] = {}
    annIds = coco.getAnnIds(imgIds=[imgid], iscrowd=None)
    anns = coco.loadAnns(annIds)
    objects = []
    for ann in anns:
        obj = {}
        bbox = ann['bbox']
        poly = np.array(ann["segmentation"][0], np.int32).reshape(4,2)
        poly = cv2.minAreaRect(poly)
        obj["poly"] = poly
        obj["category"] = CLASSES[ann["category_id"]]
        objects.append(obj)
    ret[name]["objects"] = objects
mmcv.dump(ret, "./result/gt_analy.pkl")  

loading annotations into memory...
Done (t=0.81s)
creating index...
index created!


In [14]:
pwd

'/home/xfr/git_mm/mmdetection'

In [4]:
import json
import pylab as pl
import random
import numpy as np
import cv2
import copy
import os

CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')

def get_original_name(name):
    datadir = "/home/xfr/rssid/data/val/images/"
    files = os.listdir(datadir)
    img_name = ""
    for file in files:
        img_name = file.split(".")[0]
        if(img_name == name):
            return file

def draw_all(annos, imgid, color=(0,1,0), datadir=None, have_label=True):
    img = annos['imgs'][imgid]
    filename = get_original_name(imgid)
    imgdata = cv2.imread("/home/xfr/rssid/data/val/images/"+filename)
    for obj in img['objects']:
        poly = obj['poly']
        poly = cv2.boxPoints(poly)
        loc = poly.flatten()
        xmin = min(loc[0::2])
        ymin = min(loc[1::2])
        cv2.polylines(imgdata, np.array([poly], np.int32), 1, (0, 255, 0), 1)
        ss = obj['category']
        if 'correct_catelog' in obj:
            ss = ss+'->'+obj['correct_catelog']
        if have_label:
            pass
            cv2.putText(imgdata, ss, (int(xmin),int(ymin)), 0, 0.5, color, 1)
    return imgdata

def rect_cross(rect1, rect2):
    rect = [max(rect1[0], rect2[0]),
            max(rect1[1], rect2[1]),
            min(rect1[2], rect2[2]),
            min(rect1[3], rect2[3])]
    rect[2] = max(rect[2], rect[0])
    rect[3] = max(rect[3], rect[1])
    return rect

def rect_area(rect):
    return float(max(0.0, (rect[2]-rect[0])*(rect[3]-rect[1])))

def calc_cover(rect1, rect2):
    crect = rect_cross(rect1, rect2)
    return rect_area(crect) / rect_area(rect2)

def calc_iou(r1, r2):
    area_r1 = r1[1][0] * r1[1][1]
    area_r2 = r2[1][0] * r2[1][1]
    int_pts = cv2.rotatedRectangleIntersection(r1, r2)[1]
    inter =0.0
    if int_pts is not None:
        order_pts = cv2.convexHull(int_pts, returnPoints=True)
        int_area = cv2.contourArea(order_pts)
        inter = int_area * 1.0 / (area_r1 + area_r2 - int_area + 0.0000001)
    return inter

def box_long_size(box):
    return max(box[1][0],box[1][1])

def eval_annos(annos_gd, annos_rt, iou=0.5, imgids=None, check_type=True, types=None, minscore=40, minboxsize=0, maxboxsize=400, match_same=True):
    ac_n, ac_c = 0,0
    rc_n, rc_c = 0,0
    if imgids==None:
        imgids = annos_rt.keys()
    if types!=None:
        types = { t:0 for t in types }
    miss = {"imgs":{}}
    wrong = {"imgs":{}}
    right = {"imgs":{}}
    
    for imgid in tqdm(imgids):
        v = annos_rt[imgid]
        vg = annos_gd[imgid]
        objs_g = vg["objects"]
        objs_r = v["objects"]
        bg = [obj['poly'] for obj in objs_g]
        br = [obj['poly'] for obj in objs_r]
        
        match_g = [-1]*len(bg)
        match_r = [-1]*len(br)
        if types!=None:
            for i in range(len(match_g)):
                if objs_g[i]['category'] not in types:
                    match_g[i] = -2
            for i in range(len(match_r)):
                if objs_r[i]['category'] not in types:
                    match_r[i] = -2
        for i in range(len(match_r)):
            if 'score' in objs_r[i].keys() and objs_r[i]['score']<minscore:
                match_r[i] = -2
        matches = []
        for i,boxg in enumerate(bg):
            for j,boxr in enumerate(br):
                if match_g[i] == -2 or match_r[j] == -2:
                    continue
                if match_same and objs_g[i]['category'] != objs_r[j]['category']: continue
                tiou = calc_iou(boxg, boxr)
                if tiou>iou:
                    matches.append((tiou, i, j))
        matches = sorted(matches, key=lambda x:-x[0])
        for tiou, i, j in matches:
            if match_g[i] == -1 and match_r[j] == -1:
                match_g[i] = j
                match_r[j] = i
                
        for i in range(len(match_g)):
            boxsize = box_long_size(objs_g[i]['poly'])
            erase = False
            if not (boxsize>=minboxsize and boxsize<maxboxsize):
                erase = True
            #if types!=None and not types.has_key(objs_g[i]['category']):
            #    erase = True
            if erase:
                if match_g[i] >= 0:
                    match_r[match_g[i]] = -2
                match_g[i] = -2
        
        for i in range(len(match_r)):
            boxsize = box_long_size(objs_r[i]['poly'])
            if match_r[i] != -1: continue
            if not (boxsize>=minboxsize and boxsize<maxboxsize):
                match_r[i] = -2
                    
        miss["imgs"][imgid] = {"objects":[]}
        wrong["imgs"][imgid] = {"objects":[]}
        right["imgs"][imgid] = {"objects":[]}
        miss_objs = miss["imgs"][imgid]["objects"]
        wrong_objs = wrong["imgs"][imgid]["objects"]
        right_objs = right["imgs"][imgid]["objects"]
        
        tt = 0
        for i in range(len(match_g)):
            if match_g[i] == -1:
                miss_objs.append(objs_g[i])
        for i in range(len(match_r)):
            if match_r[i] == -1:
                obj = copy.deepcopy(objs_r[i])
                obj['correct_catelog'] = 'none'
                wrong_objs.append(obj)
            elif match_r[i] != -2:
                j = match_r[i]
                obj = copy.deepcopy(objs_r[i])
                if not check_type or objs_g[j]['category'] == objs_r[i]['category']:
                    right_objs.append(objs_r[i])
                    tt+=1
                else:
                    obj['correct_catelog'] = objs_g[j]['category']
                    wrong_objs.append(obj)
                    
        
        rc_n += len(objs_g) - match_g.count(-2)
        ac_n += len(objs_r) - match_r.count(-2)
        ac_c += tt
        rc_c += tt
    print(rc_n,rc_c)
    print(ac_n, ac_c)
    if types==None:
        styps = "all"
    elif len(types)==1:
        styps = list(types.keys())[0]
    elif not check_type or len(types)==0:
        styps = "none"
    else:
        styps = "[%s, ...total %s...]"%(list(types.keys())[0], len(types))
    p = 1 if ac_n==0 else ac_c*1.0/ac_n
    r = 1 if rc_n==0 else rc_c*1.0/rc_n
    f = (2*p*r)/(p+r)
    report = "iou:%s, size:[%s,%s), types:%s, accuracy:%s, recall:%s"% (
        iou, minboxsize, maxboxsize, styps, 1 if ac_n==0 else ac_c*1.0/ac_n, 1 if rc_n==0 else rc_c*1.0/rc_n)
   
    summury = {
        "iou":iou,
        "accuracy":1 if ac_n==0 else ac_c*1.0/ac_n,
        "recall":1 if rc_n==0 else rc_c*1.0/rc_n,
        "f": f,
        "miss":miss,
        "wrong":wrong,
        "right":right,
        "report":report
    }
    return summury


In [5]:
annos_gd = mmcv.load("./result/gt_analy.pkl")
annos_rt = mmcv.load("./result/eval_analy.pkl")
summary = eval_annos(annos_gd, annos_rt, iou=0.5, types=CLASSES, minscore=0.05, minboxsize=0, maxboxsize=40000, match_same=False)


81048 52725
148945 52725


In [114]:
key_word = "miss"
for imgid in tqdm(list(summary[key_word]['imgs'].keys())):
    if(len(summary[key_word]["imgs"][imgid]["objects"]) == 0):
        continue
    img = draw_all(summary[key_word], imgid, color=(0,255,0))
    print(imgid)
    cv2.imwrite("./result/{}/{}.jpg".format(key_word, imgid), img)

P1314
P1275
P1878
P1542
P9705
P0199
P3575
P4151
P1456
P1854
P2481
P2027
P3107
P4359
P0242
P1754
P0146
P1373
P2256
P3185
P2181
P4952
P3989
P0706
P2802
P2455
P1269
P1672
P2733
P1007
P1982
P0622
P2570
P2771
P0604
P1541
P3182
P0342
P1786
P2593
P2721
P1030
P2385
P0466
P2695
P0799
P2239
P1806
P1751
P2678
P9031
P2726
P3032
P1021
P2709
P10810
P7431
P1512
P5171
P9594
P1809
P2351
P1960
P0161
P1246
P2135
P1610
P1398
P1143
P8666
P1570
P5070
P2504
P0764
P6519
P0128
P1022
P8081
P1390
P0929
P2050
P0213
P2745
P0179
P2365
P5789
P0613
P4772
P10105
P0411
P0432
P0007
P0053
P1268
P2150
P1149
P1452
P5155
P2722
P1829
P0550
P2625
P2993
P9997
P0309
P2718
P2582
P1386
P2214
P2285
P1266
P1478
P2167
P9170
P0998
P1240
P2758
P1230
P10795
P0945
P2610
P1463
P0949
P1738
P0130
P2231
P2230
P1513
P9444
P1700
P2488
P0763
P2198
P0665
P1122
P0486
P1278
P0547
P1643
P1090
P0086
P2791
P0729
P1234
P0715
P2608
P1184
P1518
P8392
P0385
P1601
P0936
P0336
P1749
P2429
P2413
P9015
P2168
P2789
P2082
P0904
P0143
P1179
P1742
P2342
P0673
P

In [113]:
annos_gd = mmcv.load("./result/gt_analy.pkl")
for name in tqdm(annos_gd.keys()):
    filename = get_original_name(name)
    img = cv2.imread("/home/xfr/rssid/data/val/images/"+filename)
    objects = annos_gd[name]["objects"]
    for obj in objects:
        poly = obj["poly"]
        poly = cv2.boxPoints(poly)
        loc = poly.flatten()
        xmin = min(loc[0::2])
        ymin = min(loc[1::2])
        cv2.polylines(img, np.array([poly], np.int32), 1, (0, 255, 0), 1)
        ss = obj['category']
        cv2.putText(img, ss, (int(xmin),int(ymin)), 0, 0.5, (0,255,0), 1)
    cv2.imwrite("./result/{}/{}.jpg".format("right", imgid), img)

In [6]:
convert = {}
for imgid in summary["wrong"]["imgs"].keys():
    objects = summary["wrong"]["imgs"][imgid]["objects"]
    for obj in objects:
        ss = obj['category']
        if ss not in convert:
            convert[ss] = {}
        if 'correct_catelog' in obj:
            mm = obj['correct_catelog']
            if mm not in convert[ss]:
                convert[ss][mm] = 0
            convert[ss][mm] += 1
for key in convert.keys():
    print(key)
    print(convert[key])

small-vehicle
{'none': 17576, 'large-vehicle': 517, 'ship': 81, 'swimming-pool': 1}
storage-tank
{'none': 1167, 'bridge': 8, 'roundabout': 3}
plane
{'none': 767, 'helicopter': 18, 'small-vehicle': 1}
harbor
{'none': 2795, 'ship': 60, 'bridge': 5, 'plane': 1}
ship
{'none': 6442, 'harbor': 35, 'small-vehicle': 119, 'large-vehicle': 15, 'swimming-pool': 2, 'tennis-court': 1}
bridge
{'none': 1120, 'harbor': 2, 'ship': 1}
tennis-court
{'none': 266, 'soccer-ball-field': 3, 'basketball-court': 3}
basketball-court
{'none': 276, 'tennis-court': 12, 'helicopter': 1, 'soccer-ball-field': 3, 'ground-track-field': 1}
baseball-diamond
{'none': 259, 'storage-tank': 1, 'tennis-court': 1}
ground-track-field
{'none': 212, 'soccer-ball-field': 3, 'baseball-diamond': 1}
roundabout
{'none': 349, 'storage-tank': 3}
large-vehicle
{'none': 8056, 'small-vehicle': 1742, 'ship': 39, 'swimming-pool': 1}
airport
{'none': 88}
helicopter
{'none': 154, 'plane': 9, 'small-vehicle': 2}
swimming-pool
{'none': 759, 'smal

In [127]:
miss = {}
for imgid in summary["miss"]["imgs"].keys():
    objects = summary["miss"]["imgs"][imgid]["objects"]
    for obj in objects:
        ss = obj['category']
        if ss not in miss:
            miss[ss] = 0
        miss[ss] += 1
for key in miss.keys():
    print(key)
    print(miss[key])

ship
1473
bridge
188
small-vehicle
22068
airport
23
swimming-pool
278
harbor
419
ground-track-field
29
large-vehicle
579
roundabout
59
storage-tank
1025
plane
259
soccer-ball-field
36
basketball-court
31
container-crane
14
tennis-court
36
helipad
2
helicopter
15
baseball-diamond
28


In [128]:
annos_gd = mmcv.load("./result/gt_analy.pkl")
right={}
for name in tqdm(annos_gd.keys()):
    objects = annos_gd[name]["objects"]
    for obj in objects:
        cls = obj["category"]
        if cls not in right:
            right[cls] = 0
        right[cls] += 1
for key in right.keys():
    print(key)
    print(right[key])


small-vehicle
50062
tennis-court
783
bridge
546
ship
13801
harbor
2443
airport
104
storage-tank
3175
baseball-diamond
273
swimming-pool
851
large-vehicle
5371
plane
2832
soccer-ball-field
152
ground-track-field
186
basketball-court
147
roundabout
228
container-crane
14
helicopter
78
helipad
2
